In [1]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib_fontja
os.chdir('../')
from src.feature import *
from src.model import *

c:\Users\takumi_inoue\projects\github\TakumiInoue0628\manufacturing-dx-purchase-prediction\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


データ準備

In [2]:
# データの読み込み
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample_submit = pd.read_csv("data/sample_submit.csv", header=None)

In [3]:
# Xとyの分割
target_col = '購入フラグ'
X_train_df = train.drop(columns=[target_col])
y_train_df = train[target_col]
X_test_df = test.copy()

特徴量エンジニアリング

In [4]:
# 学習・テストデータの結合
X_train_df.index = "train_" + X_train_df.index.astype(str)
X_test_df.index = "test_" + X_test_df.index.astype(str)
X_df = pd.concat([X_train_df, X_test_df], axis=0)

# 不要な列の削除
drop_cols = ['企業ID', '企業名']
X_df = X_df.drop(columns=drop_cols)

# 欠損値の補完
X_df = impute_missing_values(X_df)

# 指標の組み合わせによる新規指標の作成
X_df = feature_engineering(X_df)

# カテゴリ変数を数値にエンコード
categorical_cols = ['業界', '上場種別', '特徴']
X_df = onehot_encode_categorical(X_df, categorical_cols)

# ２択（はい/いいえ）変数のエンコード
binary_cols = ['アンケート６']
X_df = encode_binary(X_df, binary_cols, true=1)

# アンケートデータの特徴量化
X_df = survey_features(X_df, None)

# 組織図の特徴量化
# 特定部門の有無
org_df = org_chart_features(X_df['組織図'])
# Transformer+PCA
embedding_df = pd.read_csv("data/組織図_embed.csv", index_col=0)
transform_pca_df = pca_reduction(embedding_df, n_components=5)
transform_pca_df.columns = [f'組織図_emb_pca_{i}' for i in range(transform_pca_df.shape[1])]
X_df = pd.concat([X_df, org_df, transform_pca_df], axis=1)
X_df = X_df.drop(columns=['組織図'])

# 今後のDX展望テキストデータの特徴量化
# 文章量
length_df = text_length_features(X_df['今後のDX展望'])
# Transformer+PCA
embedding_df = pd.read_csv("data/今後のDX展望_embed.csv", index_col=0)
transform_pca_df = pca_reduction(embedding_df, n_components=5)
transform_pca_df.columns = [f'今後のDX展望_emb_pca_{i}' for i in range(transform_pca_df.shape[1])]
X_df = pd.concat([X_df, length_df, transform_pca_df], axis=1)
X_df = X_df.drop(columns=['今後のDX展望'])

# 企業概要テキストデータの特徴量化
# Transformer+PCA
embedding_df = pd.read_csv("data/企業概要_embed.csv", index_col=0)
transform_pca_df = pca_reduction(embedding_df, n_components=5)
transform_pca_df.columns = [f'企業概要_emb_pca_{i}' for i in range(transform_pca_df.shape[1])]
X_df = pd.concat([X_df, transform_pca_df], axis=1)
X_df = X_df.drop(columns=['企業概要'])

# 学習・テストデータの再分割
X_train_df = X_df.loc[X_train_df.index].reset_index(drop=True)
X_test_df = X_df.loc[X_test_df.index].reset_index(drop=True)

In [5]:
list(X_df.columns)

['従業員数',
 '事業所数',
 '工場数',
 '店舗数',
 '資本金',
 '総資産',
 '流動資産',
 '固定資産',
 '負債',
 '短期借入金',
 '長期借入金',
 '純資産',
 '自己資本',
 '売上',
 '営業利益',
 '経常利益',
 '当期純利益',
 '営業CF',
 '減価償却費',
 '運転資本変動',
 '投資CF',
 '有形固定資産変動',
 '無形固定資産変動(ソフトウェア関連)',
 'アンケート１',
 'アンケート２',
 'アンケート３',
 'アンケート４',
 'アンケート５',
 'アンケート６',
 'アンケート７',
 'アンケート８',
 'アンケート９',
 'アンケート１０',
 'アンケート１１',
 '営業利益率',
 '総資産回転率',
 'ROA',
 '従業員一人当たり営業利益',
 '流動比率',
 '無形固定資産変動率',
 '営業CFマージン',
 '投資効率対CF',
 '自己資本比率',
 '負債比率',
 '規模収益性',
 '従業員一人当たり売上',
 '負債純資産倍率',
 '投資効率対利益',
 '新規投資度合い',
 '業界_IT',
 '業界_その他',
 '業界_その他サービス',
 '業界_アパレル・美容',
 '業界_エネルギー',
 '業界_エンタメ',
 '業界_ゲーム',
 '業界_コンサルティング',
 '業界_マスコミ',
 '業界_不動産',
 '業界_人材',
 '業界_化学',
 '業界_医療・福祉',
 '業界_商社',
 '業界_外食',
 '業界_専門サービス',
 '業界_小売',
 '業界_広告',
 '業界_建設・工事',
 '業界_教育',
 '業界_機械',
 '業界_機械関連サービス',
 '業界_生活用品',
 '業界_自動車・乗り物',
 '業界_製造',
 '業界_通信',
 '業界_通信機器',
 '業界_運輸・物流',
 '業界_金融',
 '業界_電気製品',
 '業界_食品',
 '上場種別_GR',
 '上場種別_PR',
 '上場種別_ST',
 '特徴_BtoB',
 '特徴_BtoC',
 '特徴_CtoC',
 'アンケート_pos_mean',
 'アンケート_neg_mean',
 'アン

予測モデルのパラメータ最適化

In [7]:
# 基本パラメータ
lgbm_params_base = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'n_estimators': 1000,
    'is_unbalance': True,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
    }

# 探索するパラメータを定義する「関数」
def define_lgbm_params(trial):
    return {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 80),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }

# LightGBMモデルの最適化実行
lgbm_results = optimize_model(
    X_train_df,
    y_train_df,
    model_name="lightgbm",
    params_base=lgbm_params_base,
    define_params_func=define_lgbm_params,
    n_trials=50,
    early_stopping_rounds=10
)

[I 2025-11-09 04:16:14,065] A new study created in memory with name: no-name-3bdd0c03-bd1e-4395-8989-0a3107aef44e


--- Optimizing lightgbm ---


[I 2025-11-09 04:16:15,321] Trial 0 finished with value: 0.6770816847163853 and parameters: {'learning_rate': 0.03446565382258304, 'num_leaves': 74, 'max_depth': 11, 'min_child_samples': 49, 'subsample': 0.8740178121514692, 'colsample_bytree': 0.7555019155274595, 'reg_alpha': 0.0027039865674265704, 'reg_lambda': 0.5921409922484094}. Best is trial 0 with value: 0.6770816847163853.
[I 2025-11-09 04:16:16,470] Trial 1 finished with value: 0.672959067571164 and parameters: {'learning_rate': 0.03127941125893268, 'num_leaves': 53, 'max_depth': 4, 'min_child_samples': 16, 'subsample': 0.6996935238952959, 'colsample_bytree': 0.6169724368188673, 'reg_alpha': 5.744879234199783e-05, 'reg_lambda': 1.2970404308692387e-08}. Best is trial 0 with value: 0.6770816847163853.
[I 2025-11-09 04:16:17,954] Trial 2 finished with value: 0.6336712355222878 and parameters: {'learning_rate': 0.010139075350833218, 'num_leaves': 29, 'max_depth': 8, 'min_child_samples': 55, 'subsample': 0.8314567428509378, 'colsamp


--- Optimization Finished ---
Best trial for lightgbm:
  Value (Best F1 Score): 0.71883
  Best Params:
    learning_rate: 0.08582195208545372
    num_leaves: 62
    max_depth: 3
    min_child_samples: 27
    subsample: 0.6227970191813774
    colsample_bytree: 0.8064473048687641
    reg_alpha: 0.3819328462020284
    reg_lambda: 0.0009657946137515455

--- Confusion Matrix (Best Trial at Threshold: 0.4100) ---
                          |Predicted Label              |
                          |-----------------------------|
                          | Negative (0) | Positive (1) |
-----------|--------------|--------------|--------------|
True Label | Negative (0) | 497          | 66           | (TN, FP)
           |--------------|--------------|--------------|
           | Positive (1) | 42           | 137          | (FN, TP)
-----------|--------------|--------------|--------------|

--- OOF Scores (at Best Threshold) ---
  Precision: 0.67488
  Recall:    0.76536
  F1 Score:  0.71728 (F1

In [20]:
# 基本パラメータ
xgb_params_base = {
    'objective': 'binary:logistic', 
    'eval_metric': 'logloss',       
    'n_estimators': 1000,
    'scale_pos_weight': (y_train_df == 0).sum() / (y_train_df == 1).sum(), # 'is_unbalance' の代わり
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': 0,                 # 'verbose': -1 -> 'verbosity': 0
    'booster': 'gbtree'             # デフォルトだが明記
    }

# 探索するパラメータを定義する「関数」
def define_xgb_params(trial):
    return {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }

# XGBoostモデルの最適化実行
xgb_results = optimize_model(
    X_train_df,
    y_train_df,
    model_name="xgboost",
    params_base=xgb_params_base,
    define_params_func=define_xgb_params,
    n_trials=50,
    early_stopping_rounds=10
)

[I 2025-11-09 04:04:42,093] A new study created in memory with name: no-name-9b5f7f12-9d4d-4a29-a5d5-e19428595bf5


--- Optimizing xgboost ---


[I 2025-11-09 04:04:45,390] Trial 0 finished with value: 0.7126627318754044 and parameters: {'learning_rate': 0.04719749385317509, 'max_depth': 3, 'min_child_weight': 16, 'subsample': 0.9406441156107141, 'colsample_bytree': 0.7826293366735151, 'reg_alpha': 9.685945325240341e-08, 'reg_lambda': 3.924408475146515e-05}. Best is trial 0 with value: 0.7126627318754044.
[I 2025-11-09 04:04:46,147] Trial 1 finished with value: 0.3887001470465476 and parameters: {'learning_rate': 0.046220027362885927, 'max_depth': 7, 'min_child_weight': 97, 'subsample': 0.684155887276529, 'colsample_bytree': 0.7501461510686558, 'reg_alpha': 1.6297129192842832, 'reg_lambda': 0.00018670747373011117}. Best is trial 0 with value: 0.7126627318754044.
[I 2025-11-09 04:04:48,893] Trial 2 finished with value: 0.6524966237390923 and parameters: {'learning_rate': 0.040515079497189875, 'max_depth': 6, 'min_child_weight': 51, 'subsample': 0.7728595531347847, 'colsample_bytree': 0.8160470014155063, 'reg_alpha': 0.0027683924


--- Optimization Finished ---
Best trial for xgboost:
  Value (Best F1 Score): 0.72565
  Best Params:
    learning_rate: 0.014624349694125167
    max_depth: 5
    min_child_weight: 10
    subsample: 0.6122223032421644
    colsample_bytree: 0.602463210811096
    reg_alpha: 3.0208813684286765e-06
    reg_lambda: 0.020142567388587565

--- Confusion Matrix (Best Trial at Threshold: 0.4700) ---
                          |Predicted Label              |
                          |-----------------------------|
                          | Negative (0) | Positive (1) |
-----------|--------------|--------------|--------------|
True Label | Negative (0) | 502          | 61           | (TN, FP)
           |--------------|--------------|--------------|
           | Positive (1) | 42           | 137          | (FN, TP)
-----------|--------------|--------------|--------------|

--- OOF Scores (at Best Threshold) ---
  Precision: 0.69192
  Recall:    0.76536
  F1 Score:  0.72679 (F1 score on total OO

In [13]:
# 基本パラメータ
cat_params_base = {
    'objective': 'Logloss', # CatBoostでは'Logloss'を指定
        'iterations': 1000,
        'scale_pos_weight': (y_train_df == 0).sum() / (y_train_df == 1).sum(), # 不均衡データへの対処
        'random_seed': 42,
        'verbose': 0, # 学習ログを非表示
        'early_stopping_rounds': 10 # 早期停止
    }

# 探索するパラメータを定義する「関数」
def define_cat_params(trial):
    return {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True), # L2正則化
    }

# CatBoostモデルの最適化実行
cat_results = optimize_model(
    X_train_df,
    y_train_df,
    model_name="catboost",
    params_base=cat_params_base,
    define_params_func=define_cat_params,
    n_trials=50,
    early_stopping_rounds=10
)

[I 2025-11-09 03:56:18,330] A new study created in memory with name: no-name-c884303b-4f8e-4469-bc8b-6d317d928e0e


--- Optimizing catboost ---


[I 2025-11-09 03:56:23,427] Trial 0 finished with value: 0.6691326390193811 and parameters: {'learning_rate': 0.01054795785623793, 'depth': 6, 'min_data_in_leaf': 10, 'subsample': 0.7436630566839998, 'l2_leaf_reg': 0.01833593206035085}. Best is trial 0 with value: 0.6691326390193811.
[I 2025-11-09 03:56:25,494] Trial 1 finished with value: 0.6699714402284782 and parameters: {'learning_rate': 0.019843299730670023, 'depth': 4, 'min_data_in_leaf': 66, 'subsample': 0.7905692553011006, 'l2_leaf_reg': 7.72263637152063e-08}. Best is trial 1 with value: 0.6699714402284782.
[I 2025-11-09 03:56:27,576] Trial 2 finished with value: 0.7279202279202279 and parameters: {'learning_rate': 0.06322721567411647, 'depth': 3, 'min_data_in_leaf': 82, 'subsample': 0.9973031418816185, 'l2_leaf_reg': 0.27159707655587373}. Best is trial 2 with value: 0.7279202279202279.
[I 2025-11-09 03:56:30,790] Trial 3 finished with value: 0.6703269170368376 and parameters: {'learning_rate': 0.010495922769357959, 'depth': 5,


--- Optimization Finished ---
Best trial for catboost:
  Value (Best F1 Score): 0.73141
  Best Params:
    learning_rate: 0.03155510447217814
    depth: 4
    min_data_in_leaf: 16
    subsample: 0.7133641906582164
    l2_leaf_reg: 1.63545550895305

--- Confusion Matrix (Best Trial at Threshold: 0.4900) ---
                          |Predicted Label              |
                          |-----------------------------|
                          | Negative (0) | Positive (1) |
-----------|--------------|--------------|--------------|
True Label | Negative (0) | 503          | 60           | (TN, FP)
           |--------------|--------------|--------------|
           | Positive (1) | 41           | 138          | (FN, TP)
-----------|--------------|--------------|--------------|

--- OOF Scores (at Best Threshold) ---
  Precision: 0.69697
  Recall:    0.77095
  F1 Score:  0.73210 (F1 score on total OOF predictions)
  (Note: 'Best F1 Score' above is the CV mean optimized by Optuna.)

-

In [14]:
# アンサンブルモデルの学習
ensemble_result = train_ensemble_models(
    X_train_df=X_train_df,
    y_train_df=y_train_df,
    X_test_df=X_test_df,
    lgb_best_params = lgbm_results['best_params'],
    xgb_best_params = xgb_results['best_params'],
    cat_best_params = cat_results['best_params'],
    sample_submit=sample_submit,
    n_folds=5,
    early_stopping_rounds=10,
    thresholds=np.arange(0.1, 0.5, 0.01),
    random_state=42
)

--- Start Ensemble Training ---
Fold 1/5 started...
Fold 2/5 started...
Fold 3/5 started...
Fold 4/5 started...
Fold 5/5 started...
--- Ensemble Training Finished ---
Optimal Weights - LGB: 0.3333, XGB: 0.3333, CAT: 0.3333

--- Evaluation ---
Ensemble Best F1: 0.73737 (Threshold: 0.44)
LightGBM: 0.72165 (Threshold: 0.46)
XGBoost:  0.73600 (Threshold: 0.47)
CatBoost: 0.73210 (Threshold: 0.49)

--- Fold-wise Stability Evaluation (using global thresholds) ---
  Fold 1 F1 -> Ensemble: 0.78049 | LGB: 0.75000 | XGB: 0.77500 | CAT: 0.77778
  Fold 2 F1 -> Ensemble: 0.77500 | LGB: 0.61538 | XGB: 0.72222 | CAT: 0.77922
  Fold 3 F1 -> Ensemble: 0.75325 | LGB: 0.76923 | XGB: 0.72727 | CAT: 0.72000
  Fold 4 F1 -> Ensemble: 0.72941 | LGB: 0.75610 | XGB: 0.74074 | CAT: 0.72289
  Fold 5 F1 -> Ensemble: 0.63889 | LGB: 0.71429 | XGB: 0.70769 | CAT: 0.65714
Ensemble Fold F1 Mean: 0.73541, Std: 0.05152
